!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [7]:
import math
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [8]:
bertmodel, vocab = get_pytorch_kobert_model()
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /home/inmo/tide/data/emo/.cache/kobert_v1.zip
using cached model. /home/inmo/tide/data/emo/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /home/inmo/tide/data/emo/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [9]:
class BERTClassifier(nn.Module): ## 클래스를 상속
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=10,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [10]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))  

In [11]:
max_len = 512   # 텍스트 데이터 최대 길이
batch_size = 64

In [12]:
PATH = './models/'
model = torch.load(PATH+'10emotions_model_3_8epoch.pt', map_location='cpu')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model.load_state_dict(torch.load(PATH + '10emotions_model_state_dict_3_8epoch.pt', map_location='cpu'))  # state_dict를 불러 온 후, 모델에 저장

<All keys matched successfully>

In [13]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

device = torch.device('cpu')

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce GTX 1050


In [14]:
emotion_list = ['분노', '악의', '슬픔', '절망', '당황', '불안', '열등', '상처', '사랑', '편안']

In [15]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, 512, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=2)
    
    model.eval()

    for (token_ids, valid_length, segment_ids, label) in test_dataloader:
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        # for i, e in zip(out[0], emotion_list):
        #     print(f'{e}: {round(float(i),4)}')
        return out

emotion_list = ['분노', '악의', '슬픔', '절망', '당황', '불안', '열등', '상처', '사랑', '편안']

In [16]:
predict('우리의 믿음')

tensor([[-0.8537, -0.3090,  0.1065, -1.5159, -1.1381, -1.2867, -1.2769,  0.6009,
          4.5829,  1.3934]], grad_fn=<AddmmBackward0>)

### 가사집 데이터 불러오기

In [17]:
from konlpy.tag import Okt, Kkma

lyrics_df = pd.read_csv('../song/song_data/csv/learn_song_lyrics.csv', index_col=0)
lyrics_df.head(5)

,song_id,title,genre,lyrics
0,52441,너에게로 또 다시,['발라드'],그 얼마나 오랜 시간을\n 짙은 어둠에서 서성거렸나\n 내 마음을 닫아 둔채로\n ...
1,53060,솔아 솔아 푸르른 솔아,['포크/블루스'],거센 바람이 불어와서 \n 어머님의 눈물이 \n 가슴속에 사무쳐 우는 \n 갈라진 ...
2,1017150,그 아픔까지 사랑한거야,['발라드'],너를 처음 만난 날 소리없이\n 밤새 눈은 내리고\n 끝도 없이 찾아드는 기다림\n...
3,53018,향기로운 추억 (응답하라 1988 삽입곡),['발라드'],한줌 젖은 바람은 \n 이젠 희미해진 옛 추억 \n 어느 거리로 \n 날 데리고 가...
4,1859404,잊지 말아요,['성인가요/트로트'],이젠 모두 지나버린 일이야 \n 사랑했던 그 추억 마저도 \n 하지만 멀리서 \n ...


### 문장별 리스트 및 한 줄 가사 생성

In [19]:
title = '오늘도 빛나는'

lyrics = lyrics_df[lyrics_df.title.str.contains(title)].head(1).lyrics.item()
lyrics_list = [l for l in lyrics.split(' \\n ') if l != '']
lyrics = lyrics.replace('\\n', '').replace('  ', ' ')

len(lyrics), lyrics

(634,
 '별빛이 내린 밤 그 풍경 속 너와 나 날 새롭게 하는 따뜻하게 만드는 니 눈빛 니 미소 영원히 담아둘게 너로 가득한 맘 널 닮아가는 나 날 위한 선물 꿈보다 더 아름다운 서로의 품에서 끝 없는 밤을 걷자 나의 모든 날들을 다 주고싶어 내 이 맘을 모두 전하고 싶어 잠들지 못한 푸른 바람들 이렇게 밝게 이 밤을 비춰 너와 작은 일상을 함께 하는게 내 가장 큰 기쁨인걸 넌 알까 내 세상 속에 넌 빛이 되어 지금 모습 그대로 내 곁에만 행복이 짙은 날 어둠이 없는 밤 같은 맘 속에 같은 꿈이 피어난 건 우리의 정해진 운명이 맞닿은거야 나의 모든 날들을 다 주고싶어 내 이 맘을 모두 전하고 싶어 잠들지 못한 푸른 바람들 이렇게 밝게 이 밤을 비춰 너와 작은 일상을 함께 하는게 내 가장 큰 기쁨인걸 넌 알까 내 세상 속에 넌 빛이 되어 지금 모습 그대로 내 곁에만 내게 온 너란 빛이 눈 부셔도 네 앞에서 한 순간도 눈 감지 않아 다가올 시간도 계절의 바람도 널 데려가지 못하게 내가 더 좋은 사람이 되고 싶어 더 아름답게 널 안을 수 있게 잠들지 못한 잠들 수 없는 바람들이 널 부르고 있어 언제까지나 너와 함께 할거야 내 마지막 숨결도 너일거야 내 세상 속에 넌 빛이 되어 지금 모습 그대로 내 곁에만 이렇게 밝게 이 밤을 비춰 ')

### 형태소 분석을 통해 각 행마다의 값을 n으로 나누어 누적

In [ ]:
okt = Okt()
kkma = Kkma()

In [ ]:
res = predict(lyrics)[0]
pre_emo = [0]*10
s_len = len(kkma.sentences(lyrics))
for lyric in kkma.sentences(lyrics):
    emo_tensor = predict(lyric)
    for i in range(10):
        x = float(emo_tensor[0][i])/s_len
        pre_emo[i] += x
for i in range(10):
    res[i] -= round(pre_emo[i],4)

res

### 전체를 돌려버리기

In [20]:
result = predict(lyrics)[0]
for n, e in zip(emotion_list, result):
    print(n, float(e))

분노 3.6969218254089355
악의 1.2098770141601562
슬픔 -0.13585776090621948
절망 0.11310271173715591
당황 0.08569256216287613
불안 0.7442466616630554
열등 0.7030786275863647
상처 -0.04318148270249367
사랑 -2.7536795139312744
편안 -2.838031053543091


### 가사 분석 결과를 만들어보자

In [23]:
def emo_rank(pre_result):
    if pre_result[8] >= math.sqrt(sum(abs(result))/10):
        pre_result[8] = max(float(pre_result[8] ** 2), pre_result[8])
    if pre_result[9] >= math.sqrt(sum(abs(result))/10):
        pre_result[9] = max(float(pre_result[9] ** 2), pre_result[9])
        
    for i in range(8):
        pre_result[i] = (pre_result[i]/5)*4
    
    mask = sorted(enumerate(pre_result), key=lambda x:x[1], reverse=True)
    e = []
    tmp = []
    for i, x in mask:
        if len(e) >= 3:
            break
        if x > float(sum(abs(result))/len(result)):
            e.append(i+1)
        elif x >= math.sqrt(abs(sum(result)))/len(result):
            tmp.append(i+11)
    else:
        while len(e) < 3:
            if tmp:
                e.append(tmp.pop(0))
            else:
                e.append(0)
    return e

In [24]:
emo_rank_df = pd.DataFrame(columns=['song_id', 'title', '1emo','2emo','3emo'])

for i in tqdm(range(len(lyrics_df))):
    song_id = lyrics_df.loc[i, 'song_id']
    title = lyrics_df.loc[i, 'title']
    lyrics = lyrics_df.loc[i, 'lyrics'].replace('\\n', ' ')
    pre_res = predict(lyrics)
    e = emo_rank(pre_res[0])
    emo_rank_df.loc[i, :] = [song_id, title, e[0], e[1], e[2]]

emo_rank_df.to_csv('../song/lyrics_emotion_ver3.csv')

  0%|          | 0/9134 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
text = '오늘은 너무 행복한 날이었어!'

r = predict(text)
rr = emo_rank(r[0])

print(r)
rr

In [ ]:
emo_rank_df[emo_rank_df.loc[:, 'title']=='좋은 날']

In [ ]:
torch.cuda.empty_cache()

In [ ]:
result[(result > math.sqrt(sum(abs(result))/10)) & (result < sum(abs(result))/10)]

In [ ]:
(result > math.sqrt(abs(sum(result))/10))

In [ ]:
(result > sum(abs(result))/10)